# Lesson 3 Topic Modeling


Topic Models, in a nutshell, are a type of statistical language models used for uncovering hidden structure in a collection of texts. 


In a practical and more intuitively, you can think of it as a task of:


- Dimensionality Reduction, where rather than representing a text T in its feature space as {Word_i: count(Word_i, T) for Word_i in Vocabulary}, you can represent it in a topic space as {Topic_i: Weight(Topic_i, T) for Topic_i in Topics}


- Unsupervised Learning, where it can be compared to clustering, as in the case of clustering, the number of topics, like the number of clusters, is an output parameter. By doing topic modeling, we build clusters of words rather than clusters of texts. A text is thus a mixture of all the topics, each having a specific weight


Tagging, abstract “topics” that occur in a collection of documents that best represents the information in them.
There are several existing algorithms you can use to perform the topic modeling. The most common of it are, Latent Semantic Analysis (LSA/LSI), Probabilistic Latent Semantic Analysis (pLSA), and Latent Dirichlet Allocation (LDA)
In this article, we’ll take a closer look at LDA, and implement our first topic model using the sklearn.


How to evaluate LDA:

## Eye Balling Result
- Top N words
- Topics / Documents

## Intrinsic Evaluation Metrics
- Capturing model semantics
- Topics interpretability

## Human Judgements
- What is a topic
- Extrinsic Evaluation Metrics/Evaluation at task
- Is model good at performing predefined tasks, such as classification


Coherence Score
```python
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
```


In [ ]:
!pip install pyldavis

     |████████████████████████████████| 1.7 MB 7.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=c26d4a32a886bafe4a24cf3c83cff50e3fd1094e5b4fcd9420e54cdc30eb320c
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis


In [ ]:
# Load the data
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

from sklearn.datasets import fetch_20newsgroups

# categories = ['alt.atheism', 
#               'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train', 
                                  # categories=categories,
                                  shuffle=True,
                                  random_state=11)

twenty_test = fetch_20newsgroups(subset='test',
                                #  categories=categories,
                                 shuffle=True,
                                 random_state=11)


categories = twenty_test.target_names

In [ ]:
def display_topics(H, W, feature_names, no_top_words, n_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:n_top_documents]

            
sample_text = twenty_train.data[1]
sample_text

'From: chrism@cirrus.com (Chris Metcalfe)\nSubject: Nazi Eugenic Theories Circulated by CPR => (unconventianal peace)\nOrganization: Cirrus Logic Inc.\nLines: 85\n\nNow we have strong evidence of where the CPR really stands.\nUnbelievable and disgusting.  It only proves that we must\nnever forget...\n\n\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n\nIn article <1483500348@igc.apc.org> cpr@igc.apc.org (Center for Policy Research) writes:\n>\n>From: Center for Policy Research <cpr>\n>Subject: Unconventional peace proposal\n>\n>\n>A unconventional proposal for peace in the Middle-East.\n\nNot so unconventional.  Eugenic solutions to the Jewish Problem\nhave been suggested by Northern Europeans in the past.\n\n  Eugenics: a science that deals with the improvement (as by\n  control of human mating) of hereditory qualities of race\n  or breed.  -- Webster\'s Ninth Collegiate Dictionary.\n\n>5.      The emergence of a considerable number of \'mixed\'\n>marriages in Israe

In [ ]:
dir(LatentDirichletAllocation)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_approx_bound',
 '_check_feature_names',
 '_check_n_features',
 '_check_non_neg_array',
 '_check_params',
 '_e_step',
 '_em_step',
 '_get_param_names',
 '_get_tags',
 '_init_latent_vars',
 '_more_tags',
 '_perplexity_precomp_distr',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_unnormalized_transform',
 '_validate_data',
 'fit',
 'fit_transform',
 'get_params',
 'partial_fit',
 'perplexity',
 'score',
 'set_params',
 'transform']

In [ ]:
import numpy as np

docs = twenty_train.data

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(stop_words='english')
tf = tf_vectorizer.fit_transform(docs)
tf_feature_names = tf_vectorizer.get_feature_names()

n_topics = len(categories)
n_topics = 20
n_top_words = 5
n_top_documents = 5
# Run LDA
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

print("LDA Topics")
display_topics(lda_H, lda_W, tf_feature_names, n_top_words, n_top_documents)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


LDA Topics
Topic 0:
nasa space 00 gov jpl
Topic 1:
__ ___ mit lcs fr
Topic 2:
gerard alleg de7 uccxkvb dps
Topic 3:
cx w7 c_ mv uw
Topic 4:
db mov nwu een acns
Topic 5:
uchicago frank objective morality midway
Topic 6:
temple ge dane keele ocis
Topic 7:
edu subject lines organization com
Topic 8:
toronto henry udel spencer zoo
Topic 9:
uk ac liverpool liv archbishop
Topic 10:
ax max g9v b8f a86
Topic 11:
nfotis plplot virginia plot roy
Topic 12:
key file use com chip
Topic 13:
edu people com writes subject
Topic 14:
adobe smokeless nichols sherri snichols
Topic 15:
drive 55 16 ide entry
Topic 16:
radar ncr detector detectors waterloo
Topic 17:
stratus sw wpi cdt atf
Topic 18:
ncsu harris eos hernlem uoregon
Topic 19:
georgia ai uga michael athens


## Removing some data

Now let's remove some of the metadata to see if there is any improvement.

In [ ]:
remove_info = ('headers', 'footers', 'quotes'),
    
twenty_train = fetch_20newsgroups(subset='train', 
                                  remove=remove_info,
                                  categories=categories,
                                  shuffle=True,
                                  random_state=11)

twenty_test = fetch_20newsgroups(subset='test',
                                 remove=remove_info,
                                 categories=categories,
                                 shuffle=True,
                                 random_state=11)


In [ ]:

import numpy as np

docs = twenty_train.data

n_topics = 20
n_top_words = 11
n_top_documents = 5
n_features = 1000

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.80, min_df=5, 
                                max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(docs)
tf_feature_names = tf_vectorizer.get_feature_names()

n_topics = len(categories)


# Run LDA
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5, 
                                      learning_method='online', 
                                      learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

print("LDA Topics")
display_topics(lda_H, lda_W, tf_feature_names, n_top_words, n_top_documents)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


LDA Topics
Topic 0:
game team year play games ca edu season hockey win players
Topic 1:
car uiuc time just went bike left cars didn cso like
Topic 2:
cs edu information science computer list berkeley colorado faq sci pitt
Topic 3:
file window program mit files image available server ftp use code
Topic 4:
key uk chip encryption clipper ac keys security government algorithm privacy
Topic 5:
card __ speed ___ ca use drivers driver bus performance tom
Topic 6:
com netcom edu writes article jim au virginia brian fbi david
Topic 7:
gun government law state people rights guns right control states american
Topic 8:
bit using use work problem memory video com mouse time disk
Topic 9:
00 10 16 scsi 15 drive 25 apr 20 14 11
Topic 10:
windows dos graphics ibm ms pc os color washington software purdue
Topic 11:
israel jews israeli armenian turkish people armenians jewish war men said
Topic 12:
com posting host nntp access writes ca hp distribution reply article
Topic 13:
ax max g9v b8f a86 145 pl 1

In [ ]:
ng_train = fetch_20newsgroups(subset='train', 
                                  remove=remove_info,
                                  categories=categories,
                                  shuffle=True,
                                  random_state=11)

ng_train.data[-1]

'From: behanna@syl.nj.nec.com (Chris BeHanna)\nSubject: Re: Should liability insurance be required?\nOrganization: NEC Systems Laboratory, Inc.\nDistribution: usa\nLines: 32\n\nIn article <tcora-140493155620@b329-gator-3.pica.army.mil> tcora@pica.army.mil (Tom Coradeschi) writes:\n>In article <1993Apr14.125209.21247@walter.bellcore.com>,\n>fist@iscp.bellcore.com (Richard Pierson) wrote:\n>> \n>> Lets get this "No Fault" stuff straight, I lived in NJ\n>> when NF started, my rates went up, ALOT. Moved to PA\n>> and my rates went down ALOT, the NF came to PA and it\n>> was a different story. If you are sitting in a parking\n>> lot having lunch or whatever and someone wacks you guess\n>> whose insurance pays for it ? give up ?  YOURS.\n>\n>BZZZT! If it is the other driver\'s fault, your insurance co pays you, less\n>deductible, then recoups the total cost from the other guy/gal\'s company\n>(there\'s a fancy word for it, which escapes me right now), and pays you the\n>deductible. Or: you c

In [ ]:

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda_model, tf_feature_names, n_top_words)
    

Topic #0: game team year play games ca edu season hockey win players
Topic #1: car uiuc time just went bike left cars didn cso like
Topic #2: cs edu information science computer list berkeley colorado faq sci pitt
Topic #3: file window program mit files image available server ftp use code
Topic #4: key uk chip encryption clipper ac keys security government algorithm privacy
Topic #5: card __ speed ___ ca use drivers driver bus performance tom
Topic #6: com netcom edu writes article jim au virginia brian fbi david
Topic #7: gun government law state people rights guns right control states american
Topic #8: bit using use work problem memory video com mouse time disk
Topic #9: 00 10 16 scsi 15 drive 25 apr 20 14 11
Topic #10: windows dos graphics ibm ms pc os color washington software purdue
Topic #11: israel jews israeli armenian turkish people armenians jewish war men said
Topic #12: com posting host nntp access writes ca hp distribution reply article
Topic #13: ax max g9v b8f a86 145 p

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
transformed = lda_model.transform(tf)
doc_topic_dist_unnormalized = np.matrix(transformed)

# normalize the distribution (only needed if you want to work with the probabilities)
doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)

res = doc_topic_dist.argmax(axis=1).ravel().tolist()[0]

In [ ]:
doc_topic_dist

matrix([[2.17391305e-03, 2.17391305e-03, 2.17391307e-03, ...,
         2.17391305e-03, 2.17391306e-03, 2.17391313e-03],
        [4.03225813e-04, 4.03225815e-04, 4.03225817e-04, ...,
         1.03923787e-01, 3.01887726e-02, 4.03225811e-04],
        [7.46268668e-04, 7.46268670e-04, 7.46268676e-04, ...,
         2.74690253e-01, 6.07477089e-02, 7.46268661e-04],
        ...,
        [2.38095242e-03, 2.38095246e-03, 2.38095247e-03, ...,
         2.38095246e-03, 2.38095240e-03, 1.54771842e-01],
        [8.47457655e-04, 1.43464929e-01, 8.47457646e-04, ...,
         3.12270933e-01, 8.47457635e-04, 8.47457636e-04],
        [6.49350658e-04, 1.75615232e-01, 6.49350654e-04, ...,
         4.78938254e-01, 6.49350662e-04, 6.49350671e-04]])

## LDAvis

A better way to explore the LDA topics is to use pyldavis.

In [ ]:
from __future__ import division

import pandas as pd
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

docs_raw = twenty_train.data


dtm_tf = tf

In [ ]:
from pandas.compat._optional import import_optional_dependency


# def new_dep(module, errors):
#   return import_optional_dependecy("numexpr")

# import_optional_dependency = new_dep

pyLDAvis.sklearn.prepare(lda_model, dtm_tf, tf_vectorizer)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.173924  0.038494       1        1  13.553555
13     0.434907  0.290566       2        1   9.285758
14    -0.168017  0.087368       3        1   8.156238
16    -0.073416 -0.106992       4        1   7.579464
3      0.067083 -0.095247       5        1   4.882994
1     -0.104411  0.050272       6        1   4.849914
2     -0.017808 -0.054126       7        1   4.701569
9      0.230243  0.024638       8        1   4.659307
0     -0.068918  0.008425       9        1   4.629817
7     -0.142844  0.171065      10        1   4.515814
12     0.078895 -0.140015      11        1   4.504398
8     -0.040526 -0.074344      12        1   4.123575
15     0.066947 -0.204467      13        1   4.033712
19     0.081574 -0.179169      14        1   3.530742
11    -0.113132  0.214676      15        1   3.356710
18    -0.027054  0.125975      16        1   3.106528
4     -0.012105  0.038313      17        1   2.966381
5     -0.028530 -0.098924      18        1   2.760105
6     -0.102704  0.047037      19        1   2.713186
10     0.113741 -0.143546      20        1   2.090234, topic_info=        Term          Freq         Total Category  logprob  loglift
143       ax  64325.000000  64325.000000  Default  30.0000  30.0000
346      edu  20925.000000  20925.000000  Default  29.0000  29.0000
241      com  12135.000000  12135.000000  Default  28.0000  28.0000
588      max   4796.000000   4796.000000  Default  27.0000  27.0000
991   writes   7713.000000   7713.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
872  support    127.683464   1217.690982  Topic20  -4.9654   1.6127
346      edu    273.788574  20925.758089  Topic20  -4.2026  -0.4685
202       cc    128.755397   1349.539019  Topic20  -4.9571   1.5183
950       ve    145.385546   2598.669343  Topic20  -4.8356   0.9845
16        16    122.514143   1820.883647  Topic20  -5.0067   1.1690

[871 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         3  0.001967    00
0         5  0.033443    00
0         8  0.954768    00
0        14  0.003934    00
0        16  0.002623    00
...     ...       ...   ...
995      15  0.015418  year
995      16  0.112744  year
995      18  0.004336  year
995      19  0.006264  year
995      20  0.000482  year

[5236 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 14, 15, 17, 4, 2, 3, 10, 1, 8, 13, 9, 16, 20, 12, 19, 5, 6, 7, 11])

In [ ]:
!pip install --upgrade pandas

In [ ]:
pyLDAvis.sklearn.prepare(lda_model, dtm_tf, tf_vectorizer, mds='tsne')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:348: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  error = np.finfo(np.float).max
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
1      -10.878068   10.538728       1        1  9.590226
14     -60.884026  -75.591820       2        1  8.926920
17      -2.336119  -53.289074       3        1  8.648744
0      -61.385319  -12.914634       4        1  8.592349
2      -28.737743  126.296997       5        1  8.413419
7        1.066606 -123.552811       6        1  7.310783
11      45.140720   -7.947869       7        1  6.744548
9     -140.795975  -98.530548       8        1  6.304130
3       61.531681  -83.240562       9        1  5.777927
19     -72.563057 -140.396881      10        1  5.166381
8       15.984491   63.578186      11        1  4.370576
4     -160.851624   68.945145      12        1  3.497923
5       85.961830   51.922935      13        1  3.476978
18     -99.020836  109.936348      14        1  2.767902
10    -176.764130   -8.681684      15        1  2.106676
12    -117.189880  -37.420200      16        1  1.999123
16     108.829918  -22.946203      17        1  1.940902
13      47.588097  121.934814      18        1  1.585996
15    -106.467834   30.176889      19        1  1.392244
6      -50.613804   59.328842      20        1  1.386252, topic_info=           Term         Freq        Total Category  logprob  loglift
143        bike  1026.000000  1026.000000  Default  30.0000  30.0000
346  experience  1285.000000  1285.000000  Default  29.0000  29.0000
241      copies   626.000000   626.000000  Default  28.0000  28.0000
991      wouldn   373.000000   373.000000  Default  27.0000  27.0000
427     happens   299.000000   299.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
906     thought     7.671541   113.305954  Topic20  -4.9963   1.5860
527      letter    12.977903   356.776303  Topic20  -4.4705   0.9647
311          dr    10.711975   237.695337  Topic20  -4.6624   1.1789
517      launch    11.185642   448.797594  Topic20  -4.6191   0.5866
153        body     8.401991   165.517746  Topic20  -4.9053   1.2980

[864 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         8  0.959879       00
0         9  0.031300       00
0        10  0.005217       00
1         1  0.112840      000
1         3  0.021157      000
...     ...       ...      ...
995      11  0.003428  xfree86
995      13  0.003428  xfree86
995      14  0.068554  xfree86
995      17  0.003428  xfree86
995      20  0.003428  xfree86

[4086 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 15, 18, 1, 3, 8, 12, 10, 4, 20, 9, 5, 6, 19, 11, 13, 17, 14, 16, 7])

## Excercise

Explore different parameters for the LDA model and visualize the results. Create a new pipline and experiment with HashVectorizer instehad of CounterVectorizer.

In [ ]:
pyLDAvis.sklearn.prepare(lda_model, dtm_tf, tf_vectorizer, mds='tsne')


# Non-Negative Matrix Factorization


In [ ]:
# Importing Necessary packages

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [ ]:
# Let's check the first 3 articles
text_data= fetch_20newsgroups(remove=('headers', 'footers', 'quotes')).data
text_data[:3]

['I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.',
 "A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't an

In [ ]:
# converting the given text term-document matrix
 
vectorizer = TfidfVectorizer(max_features=1500, min_df=10, stop_words='english')
X = vectorizer.fit_transform(text_data)
words = np.array(vectorizer.get_feature_names())

# The algorithm splits each term in the document and assigns weightage to each words.
print(X)
print("X = ", words)

  (0, 829)	0.13596515131134768
  (0, 809)	0.1439640091285723
  (0, 707)	0.16068505607893963
  (0, 672)	0.16927150728890597
  (0, 1495)	0.1274990882101728
  (0, 506)	0.19413995565094086
  (0, 887)	0.17648781190400797
  (0, 757)	0.09424560560725692
  (0, 247)	0.17513150125349702
  (0, 1158)	0.1651151431885443
  (0, 1218)	0.19781957502373113
  (0, 128)	0.190572546028195
  (0, 1256)	0.153503242191245
  (0, 1118)	0.12154002727766956
  (0, 273)	0.14279390121865662
  (0, 484)	0.1714763727922697
  (0, 767)	0.18711856186440218
  (0, 808)	0.18303366583393096
  (0, 469)	0.2009979730339519
  (0, 411)	0.14249215589040326
  (0, 1191)	0.17201525862610714
  (0, 278)	0.630558141606117
  (0, 1472)	0.1855076564575762
  (1, 1355)	0.12138696862814867
  (1, 653)	0.1728163048656526
  :	:
  (11312, 1027)	0.45507155319966874
  (11312, 647)	0.21811161764585577
  (11312, 1302)	0.2391477981479836
  (11312, 1276)	0.39611960235510485
  (11312, 1100)	0.1839292570975713
  (11312, 926)	0.2458009890045144
  (11312, 140


Let's apply NMF to our data and view the topics generated. For simplicity, we will look at 10 topics that the model has generated. 

A commonly used method of optimization is the multiplicative update method. In this method, W and H are each updated iteratively according to the following rule:


img.png



img2.png

it's possible to find an implementation in Scikits-learn.


In [ ]:
# Applying Non-Negative Matrix Factorization
 
nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-10:]]])))

/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kwargs['lwork'] = ret[-2][0].real.astype(numpy.int)
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

Topic 1: way,really,ve,time,good,know,think,like,just,don
Topic 2: info,help,looking,card,hi,know,advance,mail,does,thanks
Topic 3: church,does,christians,christian,faith,believe,christ,bible,jesus,god
Topic 4: league,win,hockey,play,players,season,year,games,team,game
Topic 5: bus,floppy,card,controller,ide,hard,drives,disk,scsi,drive
Topic 6: 20,price,condition,shipping,offer,space,10,sale,new,00
Topic 7: problem,running,using,use,program,files,window,dos,file,windows
Topic 8: public,nsa,algorithm,escrow,government,keys,clipper,encryption,chip,key
Topic 9: state,war,turkish,armenians,jews,armenian,israeli,government,israel,people
Topic 10: email,internet,pub,ftp,article,com,university,cs,soon,edu


When we decompose the representation into two matrices similar words will be close to each other. The word “eat” would be likely to appear in food-related articles, and therefore co-occur with words like “tasty” and “food”. Therefore, these words would probably be grouped together into a “food” component vector, and each article would have a certain weight of the “food” topic.
Therefore, an NMF decomposition of the term-document matrix would yield components that could be considered “topics”, and decompose each document into a weighted sum of topics. This is called topic modeling and is an important application of NMF.

This is another example where the underlying components (topics) and their weights should be non-negative.
Another interesting property of NMF is that it naturally produces sparse representations. 

In [ ]:
print(H[:10,:10])

[[1.31208864e-17 1.04980266e-02 2.91670348e-05 9.86718395e-03
  5.48541126e-07 1.12428230e-05 1.65990695e-08 8.03079116e-06
  1.74395097e-69 1.40609464e-54]
 [3.31342076e-12 0.00000000e+00 2.26990130e-09 7.75868372e-12
  2.75045437e-03 5.18073220e-04 5.61954125e-04 1.42645853e-09
  4.57317201e-29 5.87146468e-37]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.79823512e-11 4.36444161e-02 1.69893233e-03 2.39791071e-03
  2.30176212e-03 1.95076098e-06 2.26562658e-03 2.12086657e-04
  3.57527654e-08 1.05442577e-07]
 [1.61778381e-13 3.13363148e-03 0.00000000e+00 9.27300513e-03
  0.00000000e+00 0.00000000e+00 4.30527068e-03 0.00000000e+00
  4.70932594e-11 9.29118673e-13]
 [9.96290280e-01 2.43899633e-01 8.04519710e-02 5.34003291e-02
  3.62990056e-02 7.33490127e-02 4.62836078e-02 4.26898049e-02
  4.64490387e-03 2.50991882e-03]
 [0.00000000e+00 0.00000000e+00 2.16851335e-02 0.00000000e

In [ ]:
print(W[:10,:10])

[[2.95859362e-02 2.72258526e-02 0.00000000e+00 1.93189001e-03
  0.00000000e+00 1.66023322e-02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.15936887e-03 2.98289363e-02 9.39272124e-09 4.72059329e-04
  3.17006059e-02 7.90012578e-03 0.00000000e+00 5.53289948e-03
  8.99232476e-10 9.29872564e-04]
 [6.46701075e-02 6.02117069e-02 0.00000000e+00 7.97196091e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.29472667e-02]
 [4.41411386e-03 2.80098839e-02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.18615377e-02 0.00000000e+00 8.90132800e-02
  0.00000000e+00 1.58118696e-12]
 [3.43132847e-02 1.44606630e-03 3.26298861e-03 0.00000000e+00
  0.00000000e+00 2.35394437e-02 6.73479023e-03 0.00000000e+00
  0.00000000e+00 1.31616865e-02]
 [1.64781595e-02 0.00000000e+00 3.81679386e-03 0.00000000e+00
  1.09976358e-10 4.09447617e-03 8.15131860e-03 0.00000000e+00
  1.56674926e-02 0.00000000e+00]
 [6.33472678e-03 6.45342347e-02 1.45985595e-04 2.73556175e

The we may want to impose stronger sparsity constraints or prevent the weights from becoming too large. To solve these problems, we can introduce L1 and L2 regularization losses on the weights of the matrices

# LSA (Latent Semantic Analysis)


Latent Semantic Analysis, or LSA, is one of the foundational techniques in topic modeling. The core idea is to take a matrix of what we have — documents and terms — and decompose it into a separate document-topic matrix and a topic-term matrix.

Latent Semantic Analysis, or LSA, is one of the foundational techniques in topic modeling. The core idea is to take a matrix of what we have — documents and terms — and decompose it into a separate document-topic matrix and a topic-term matrix.

The first step is generating our document-term matrix. Given m documents and n words in our vocabulary, we can construct an m × n matrix A in which each row represents a document and each column represents a word. In the simplest version of LSA, each entry can simply be a raw count of the number of times the j-th word appeared in the i-th document. In practice, however, raw counts do not work particularly well because they do not account for the significance of each word in the document. For example, the word “nuclear” probably informs us more about the topic(s) of a given document than the word “test.”

Consequently, LSA models typically replace raw counts in the document-term matrix with a tf-idf score. Tf-idf, or term frequency-inverse document frequency, assigns a weight for term j in document i as follows:


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
  
# tf-idf matrix: 
vectorizer =  # complete here


In [ ]:
# SVD to reduce dimensionality: 
svd_model = # complete here



In [ ]:
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = # complete here

In [ ]:
svd_matrix = svd_transformer.fit_transform(text_data)


In [ ]:
# svd_matrix can later be used to compare documents, compare words, or compare queries with documents
# using for example cosine similarity
svd_matrix

# Comparison

In [ ]:

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

data, _ = fetch_20newsgroups(shuffle=True, 
                             random_state=1,
                             remove=('headers', 'footers', 'quotes'),
                             return_X_y=True)
data_samples = data[:n_samples]

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data_samples)

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(data_samples)
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
Extracting tf features for LDA...

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,



Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope program mac email home contact blood
Topic #6: file problem files format win sound ftp pub read save site help image available create copy running memory self version
Topic #7: game team games y

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


done in 4.199s.

Topics in LDA model:
Topic #0: edu com mail send graphics ftp pub available contact university list faq ca information cs 1993 program sun uk mit
Topic #1: don like just know think ve way use right good going make sure ll point got need really time doesn
Topic #2: christian think atheism faith pittsburgh new bible radio games alt lot just religion like book read play time subject believe
Topic #3: drive disk windows thanks use card drives hard version pc software file using scsi help does new dos controller 16
Topic #4: hiv health aids disease april medical care research 1993 light information study national service test led 10 page new drug
Topic #5: god people does just good don jesus say israel way life know true fact time law want believe make think
Topic #6: 55 10 11 18 15 team game 19 period play 23 12 13 flyers 20 25 22 17 24 16
Topic #7: car year just cars new engine like bike good oil insurance better tires 000 thing speed model brake driving performance
Topic

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
